In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.vectorstores import FAISS

from langchain.embeddings import HuggingFaceBgeEmbeddings 
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA


In [2]:
##Read the pdfs from folder
loader = PyPDFDirectoryLoader('./us_census')

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)

final_documents = text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3')

In [3]:
len(final_documents)

316

In [4]:
# Embedding using HuggingFace
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    # model_name = 'microsoft/Phi-3-mini-4k-instruct',
    model_name = 'BAAI/bge-small-en-v1.5',
    model_kwargs = {'device': 'cpu'},
    encode_kwargs = {'normalize_embeddings': True}
)

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence_transformers`.

In [12]:
import numpy as np
np.array(huggingface_embeddings.embed_query(final_documents[0].page_content))
np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape

NameError: name 'huggingface_embeddings' is not defined

In [14]:
#VectorStore Creation
vectorstore = FAISS.from_documents(final_documents[:120], huggingface_embeddings)

NameError: name 'huggingface_embeddings' is not defined

In [15]:
#Query using Similarity Search
query = 'What is insurance coverage?'
relevant_documnets = vectorstore.similarity_search(query)

relevant_documnets[0].page_content

NameError: name 'vectorstore' is not defined

In [16]:
retriever = vectorstore.as_retriever(search_type = 'similarity', search_kwargs = {'k': 3})
retriever

NameError: name 'vectorstore' is not defined

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']="HF_API_KEY"

In [ ]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

In [ ]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

In [ ]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
"""

In [ ]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [ ]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [ ]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [ ]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])